In [ ]:
# thi file contains all preprocessing and mysql upload for the initial database state upload
# data is uploaded from mysql to bigquery

### Data Preprocessing

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd.set_option('display.max_columns',500) 
plt.style.use('seaborn-v0_8-colorblind') #a style that can be used for plots
sns.set_style('whitegrid')

In [ ]:
df = pd.read_csv('data/heart_disease_uci.csv', index_col = 0, header = 0)
df.info()

In [ ]:
# drop dataset the data is from 
# and drop 'ca' - we are missing > 60% of this values and we dont need to fill it in
df = df.drop(['dataset', 'ca'], axis=1)

In [ ]:
# rename columns for my sanity
df.rename(columns = {'cp':'Chest_Pain_Type', 'trestbps':'Resting_Blood_Pressure','chol':'serum_cholesterol','fbs':'Fasting_Blood_Sugar','restecg':'Resting_Electrocardiographic','thalch':'Maximum_Heart_Rate','exang':'Exercise_Induced_Angina','oldpeak':'ST_Depression_Induced','thal':'Heart_Condition'}, inplace = True)

In [ ]:
# fill Resting_Blood_Pressure n/a with mean values
df["Resting_Blood_Pressure"].fillna(df["Resting_Blood_Pressure"].mean(), inplace=True)

# fill serum_cholesterol n/a with mean values
df["serum_cholesterol"].fillna(df["serum_cholesterol"].mean(), inplace=True)

# fill age n/a with mean values
df["age"].fillna(df["age"].mean(), inplace=True)

# fill Fasting_Blood_Sugar n/a with mean valuesForst. 
df["Fasting_Blood_Sugar"].fillna(df["Fasting_Blood_Sugar"].astype(float).mean(), inplace=True)

# fill Maximum_Heart_Rate n/a with mean values
df["Maximum_Heart_Rate"].fillna(df["Maximum_Heart_Rate"].mean(), inplace=True)

# fill ST_Depression_Induced n/a with mean values
df["ST_Depression_Induced"].fillna(df["ST_Depression_Induced"].mean(), inplace=True)

In [ ]:
# function to help fill in categorical data
def na_randomfill(series):
    na_mask = pd.isnull(series)   # boolean mask for null values
    n_null = na_mask.sum()        # number of nulls in the Series
    
    if n_null == 0:
        return series             # if there are no nulls, no need to resample
    
    # Randomly sample the non-null values from our series
    #  only sample this Series as many times as we have nulls 
    fill_values = series[~na_mask].sample(n=n_null, replace=True, random_state=0)

    # This ensures our new values will replace NaNs in the correct locations
    fill_values.index = series.index[na_mask]
    
    return series.fillna(fill_values) 

In [ ]:
# fill in the rest of the values with distributed random values
df["Resting_Electrocardiographic"] = na_randomfill(df["Resting_Electrocardiographic"])
df["Exercise_Induced_Angina"] = na_randomfill(df["Exercise_Induced_Angina"])
df["ST_Depression_Induced"] = na_randomfill(df["ST_Depression_Induced"])
df["slope"] = na_randomfill(df["slope"])
df["Heart_Condition"] = na_randomfill(df["Heart_Condition"])

### Generate Data/Connect and Load Data to MySQL

In [ ]:
#!pip install names

In [ ]:
#!pip install random_address

In [ ]:
import names
import random
import random_address
from datetime import date
import datetime

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  port=3306,
  password="mQ0NjNmZ!",
  database="cardiology"
)

In [ ]:
mycursor = mydb.cursor()

In [ ]:
#random number generator
def rand_x_digit_num(x, leading_zeroes=True):
    """Return an X digit number, leading_zeroes returns a string, otherwise int"""
    if not leading_zeroes:
        # wrap with str() for uniform results
        return random.randint(10**(x-1), 10**x-1)  
    else:
        if x > 6000:
            return ''.join([str(random.randint(0, 9)) for i in xrange(x)])
        else:
            return '{0:0{x}d}'.format(random.randint(0, 10**x-1), x=x)      